<a href="https://colab.research.google.com/github/marissa-graham/deep_learning/blob/master/Lab_7_(Transformer_GC_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget -O ./text_files.tar.gz 'http://liftothers.org/dokuwiki/lib/exe/fetch.php?media=cs501r_f2018:es-en-general-conference.tar.gz' 
!tar -xzf text_files.tar.gz

!pip3 install torch spacy torchtext==0.2.3
!python -m spacy download en
!python -m spacy download es

--2018-10-21 05:01:38--  http://liftothers.org/dokuwiki/lib/exe/fetch.php?media=cs501r_f2018:es-en-general-conference.tar.gz
Resolving liftothers.org (liftothers.org)... 50.62.229.1
Connecting to liftothers.org (liftothers.org)|50.62.229.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18318204 (17M) [application/octet-stream]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[===================>]  17.47M  7.06MB/s    in 2.5s    

2018-10-21 05:01:40 (7.06 MB/s) - ‘./text_files.tar.gz’ saved [18318204/18318204]


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')


    Linking successful
    /usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/es

    You can now load the model via spacy.load('es')



In [0]:
import glob
import csv

import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

from torchtext import data, datasets
import spacy

import numpy as np
import math
import copy
import time

print(glob.glob('./*'))

['./sample_data', './en-es_gc_2010-2017_en.txt', './text_files.tar.gz', './en-es_gc_2010-2017_es.txt', './en-es_conference.csv']


In [0]:
gc_data_txt = ['./en-es_gc_2010-2017_en.txt', './en-es_gc_2010-2017_es.txt']
gc_data_csv = './en-es_conference.csv'

all_rows = []
with open(gc_data_csv) as f:
    i = 0
    reader = csv.reader(f)
    for row in reader:
        i += 1
        #all_rows.append([row[0], row[1], row[2]])
        print(row)
        if i > 15:
            break

['', 'en', 'es']
['0', 'The temple holds a place at the very center of our most sacred beliefs, and the Lord asks that we attend, ponder, study, and find personal meaning and application individually.', 'El templo tiene un lugar en el centro mismo de nuestras creencias más sagradas y el Señor nos pide que asistamos, meditemos, estudiemos y encontremos significado personal y aplicación individual.']
['1', 'We will come to understand that through the ordinances of the temple, the power of godliness is manifest in our lives and that because of temple ordinances, we can be armed with God’s power, and His name will be upon us, His glory round about us, and His angels have charge over us.', 'Llegaremos a entender que mediante las ordenanzas del templo, el poder de la divinidad se manifiesta en nuestra vida y que, gracias a las ordenanzas del templo, podemos estar armados con el poder de Dios y Su nombre estará sobre nosotros, Su gloria nos rodeará y Sus ángeles nos guardarán.']
['2', 'I wond

# Model architecture and stuff from before

In [0]:
# Called in Encoder, Decoder, EncoderLayer, DecoderLayer, MultiHeadedAttention
def clones(module, N):
    """Produce N identical copies of a layer (for multi-headed attention)."""
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

# Called by MultiHeadedAttention
def attention(query, key, value, mask=None, dropout=0.0):
    """Compute scaled dot product attention."""
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    
    # jupyter: dropout default is None, and this line becomes
    # if dropout is not None: p_attn = dropout(p_attn)
    # Effectively does the same thing
    p_attn = F.dropout(p_attn, p=dropout)
    return torch.matmul(p_attn, value), p_attn

# Used to generate predictions
class Generator(nn.Module):
    """Standard linear + softmax generation step."""
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

# Standard piece of the model; called by make_model
class PositionwiseFeedForward(nn.Module):
    """Two linear transformations with a ReLU in between."""
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        # Torch linears have a `b` by default. 
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

# Allow the model to pay attention to relative positions of tokens
class PositionalEncoding(nn.Module):
    """
    Sinusoid-based positional encoding to allow the model to extrapolate to 
    sequence lengths longer than those seen in training.
    
    One sinusoid of a different frequency for each dimension in the embedding.
    """
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        
        pe = torch.zeros(max_len, d_model) # 2d
        #position = torch.arange(0, max_len).unsqueeze(1) 
        #div_term = torch.exp(torch.arange(0, d_model, 2) *
        #                     -(math.log(10000.0) / d_model))
        position = torch.arange(0, max_len).unsqueeze(1).double()
        div_term = torch.exp(torch.arange(0, d_model, 2).double() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        """
        pe = np.zeros((max_len, d_model))
        position = np.arange(0, max_len).reshape((max_len, 1))
        div_term = np.exp(np.arange(0, d_model, 2) * -np.log(10000.0/d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        pe = torch.tensor(pe, dtype=torch.long).unsqueeze(0)
        """
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)#.float()
        return self.dropout(x)

# Learned embedding model for conversion of input/output tokens.
class Embeddings(nn.Module):
    
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        # Scale by sqrt(d_model) (to match the scaled dot product?)
        return self.lut(x) * math.sqrt(self.d_model)
    
class MultiHeadedAttention(nn.Module):
    
    def __init__(self, h, d_model, dropout=0.1):
        """Take in model size and number of heads."""
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.p = dropout
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
                             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.p)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)
    
class LayerNorm(nn.Module):
    """Basic layer normalization."""
    
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2*(x - mean)/(std + self.eps) + self.b_2
    
class SublayerConnection(nn.Module):
    """A residual (skip) connection followed by a layer norm."""
    
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        # Apply residual connection to any sublayer with the same size
        return x + self.dropout(sublayer(self.norm(x)))

# This gets fed to an Encoder class
class EncoderLayer(nn.Module):
    """An encoder layer consists of self-attention and feed-forward."""
    
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        """Follow Figure 1 in the paper for connections."""
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)
    
class Encoder(nn.Module):
    """The main encoder is a stack of N encoder layers."""
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)
    
class DecoderLayer(nn.Module):
    """
    A decoder layer consists of self-attention, source-attention, and 
    feed-forward.
    """
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)
    
class Decoder(nn.Module):
    """The main decoder is a stack of N decoder layers."""
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)
    
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
    
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, 
               dropout=0.1):
    """
    Put all the pieces of the model together, given the source and target 
    vocabularies, number of layers for the encoder and decoder, dimensions for
    the model embedding, dimensions for feed-forward, number of heads to use 
    for multi-headed attention, and dropout parameter.
    """
    
    # Convenience definition to make copies of things
    c = copy.deepcopy
    
    # Attention, feed-forward, & positional encoding to feed Encoder and Decoder
    attn = MultiHeadedAttention(h, d_model, dropout)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # Important from their (whose?) code. Initialize params w/ Glorot or fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

# Note: This part is incredibly important. 
# Need to train with this setup or the model is very unstable.

# Called by get_std_opt to make model_opt in main
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup**(-1.5)))
    
# Used to make 'criterion' in main
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        
        # If the mask is not empty
        if mask.nelement() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

# Make the batches to iterate over for training and validation
class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))
                
# Called in main to make 'model_opt'
def get_std_opt(model):
    """ Set up default parameters for the Noam Optimizer. """
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), 
                             eps=1e-9))

# Called by make_std_mask 
def subsequent_mask(size):
    """
    Mask out subsequent positions to ensure predictions for position i can only
    depend on the known outputs at positions less than i.
    """
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0


# Called to set up the MyIterator things
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    """
    Ensure that the batch size padded to the maximum batchsize does not
    surpass a threshold.
    """
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)

class Batch:
    def __init__(self, src, trg, src_mask, trg_mask, ntokens):
        self.src = src
        self.trg = trg
        self.src_mask = src_mask
        self.trg_mask = trg_mask
        self.ntokens = ntokens
        
# Called by data_gen and rebatch
def make_std_mask(src, tgt, pad):
    """ Hide future words and batch padding. """
    src_mask = (src != pad).unsqueeze(-2)
    tgt_mask = (tgt != pad).unsqueeze(-2)
    tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(
        tgt_mask.data))
    return src_mask, tgt_mask

# Called on the stuff in MyIterator at each epoch
def rebatch(pad_idx, batch):
    """
    Ensure that we have very evenly divided batches with minimal padding
    (despite variations in sentence length).
    """
    src, trg = batch.src.transpose(0, 1), batch.trg.transpose(0, 1)
    src_mask, trg_mask = make_std_mask(src, trg, pad_idx)
    return Batch(src, trg, src_mask, trg_mask, (trg[1:] != pad_idx).data.sum())

# Loss function (compute each timestep separately to optimize memory)
def loss_backprop(generator, criterion, out, targets, normalize):
    
    assert out.size(1) == targets.size(1)
    total = 0.0
    out_grad = []
    for i in range(out.size(1)):
        out_column = Variable(out[:, i].data, requires_grad=True)
        gen = generator(out_column)
        loss = criterion(gen, targets[:, i]) / normalize.float()
        total += loss.data[0]
        loss.backward()
        out_grad.append(out_column.grad.data.clone())
    out_grad = torch.stack(out_grad, dim=1)
    out.backward(gradient=out_grad)
    return total

def train_epoch(train_iter, model, criterion, opt, transpose=False):
    model.train()
    for i, batch in enumerate(train_iter):
        src, trg, src_mask, trg_mask = \
            batch.src, batch.trg, batch.src_mask, batch.trg_mask
        out = model.forward(src, trg[:, :-1], src_mask, trg_mask[:, :-1, :-1])
        loss = loss_backprop(model.generator, criterion, out, trg[:, 1:], 
                             batch.ntokens) 
                        
        model_opt.step()
        model_opt.optimizer.zero_grad()
        if i % 10 == 1:
            print("Batch", i, "Loss", np.round(loss.item(),4), 
                 "Learning Rate", np.round(model_opt._rate,8))
            
def valid_epoch(valid_iter, model, criterion, transpose=False):
    model.eval()
    total = 0
    for i, batch in enumerate(valid_iter):
        src, trg, src_mask, trg_mask = \
            batch.src, batch.trg, batch.src_mask, batch.trg_mask
        out = model.forward(src, trg[:, :-1], src_mask, trg_mask[:, :-1, :-1])
        loss = loss_backprop(model.generator, criterion, out, trg[:, 1:], 
                             batch.ntokens) 
        print("Batch", i, "validation loss:", loss.item())

# Different for GC dataset

In [0]:
# Set up tokenizing of input--modify slightly for en/es vs. en/de
spacy_es = spacy.load('es')
spacy_en = spacy.load('en')

def tokenize_es(text):
    return [tok.text for tok in spacy_es.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

#for i in range(10):
#    print(tokenize_es(all_rows[i][2]), tokenize_en(all_rows[i][1]))

In [0]:
# Set up a few special tokens
BOS_WORD = '<s>'
EOS_WORD = '</s>'
BLANK_WORD = "<blank>"

# data is a torchtext module, remember
SRC = data.Field(tokenize=tokenize_es, pad_token=BLANK_WORD)
TGT = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, 
                 eos_token = EOS_WORD, pad_token=BLANK_WORD)

gc_data = data.TabularDataset(gc_data_csv, 'csv',
                              fields=[('Index', None),('trg', TGT),('src', SRC)],
                              skip_header=True)

In [0]:
print(gc_data[0])
print(gc_data[0].src, gc_data[0].trg)
train, val = gc_data.split(split_ratio=0.99)
print(len(gc_data))
print(len(train), ":", len(val))

MAX_LEN = 100
MIN_FREQ = 1

SRC.build_vocab(train.src, min_freq=MIN_FREQ)
TGT.build_vocab(train.trg, min_freq=MIN_FREQ)

print(len(SRC.vocab))
print(len(TGT.vocab))

# Set up a few constant parameters
n_src = len(SRC.vocab)
n_tgt = len(TGT.vocab)
pad_idx = TGT.vocab.stoi["<blank>"]

['El', 'templo', 'tiene', 'un', 'lugar', 'en', 'el', 'centro', 'mismo', 'de', 'nuestras', 'creencias', 'más', 'sagradas', 'y', 'el', 'Señor', 'nos', 'pide', 'que', 'asistamos', ',', 'meditemos', ',', 'estudiemos', 'y', 'encontremos', 'significado', 'personal', 'y', 'aplicación', 'individual', '.'] ['The', 'temple', 'holds', 'a', 'place', 'at', 'the', 'very', 'center', 'of', 'our', 'most', 'sacred', 'beliefs', ',', 'and', 'the', 'Lord', 'asks', 'that', 'we', 'attend', ',', 'ponder', ',', 'study', ',', 'and', 'find', 'personal', 'meaning', 'and', 'application', 'individually', '.']
105786
104728 : 1058
46989
32037


In [0]:
# Batch size is 4096 in the colab notebook and 12000 in the jupyter
BATCH_SIZE = 4096

# Set up the model 
model = make_model(n_src, n_tgt, N=6)
model_opt = get_std_opt(model)
model.cuda()

# Set up the label smoothing to penalize overconfidence
criterion = LabelSmoothing(size=n_tgt, padding_idx=pad_idx, smoothing=0.1)
criterion.cuda()

# Set up training and validation iterators
train_iter = MyIterator(train, batch_size=BATCH_SIZE, device=0,
                        repeat=False, sort_key=lambda x: (len(x.src), 
                                                          len(x.trg)),
                        batch_size_fn=batch_size_fn, train=True)

valid_iter = MyIterator(val, batch_size=BATCH_SIZE, device=0,
                        repeat=False, sort_key=lambda x: (len(x.src), 
                                                          len(x.trg)),
                        batch_size_fn=batch_size_fn, train=False)

num_train_batches = 0
for i, batch in enumerate(train_iter):
    num_train_batches += 1
num_valid_batches = 0
for i, batch in enumerate(valid_iter):
    num_valid_batches += 1
print("\nNumber of training batches:", num_train_batches)
print("Number of validation batches:", num_valid_batches, "\n")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:263: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Number of training batches: 781
Number of validation batches: 10 



/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


In [0]:
# Train the model
for epoch in range(15):
    print("\n\n%%%%%%%%%% EPOCH " + str(epoch) + " %%%%%%%%%%\n")
    train_epoch((rebatch(pad_idx, b) for b in train_iter), model, criterion, 
                model_opt)
    valid_epoch((rebatch(pad_idx, b) for b in valid_iter), model, criterion)



%%%%%%%%%% EPOCH 0 %%%%%%%%%%



RuntimeError: ignored

In [0]:
# Predict a translation using greedy decoding for simplicity
# You must use the jupyter EncoderDecoder class to run this!
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], 
                       dim=1)
    return ys

# Decode the model to produce translations (first sentence in validation set)
model.eval()
for param in model.parameters():
    param.requires_grad = False
    
print(TGT.vocab)

for i, batch in enumerate(valid_iter):
    
    src = batch.src.transpose(0, 1)[:1]
    src_mask = (src != SRC.vocab.stoi["<blank>"]).unsqueeze(-2)
    out = greedy_decode(model, src, src_mask, 
                        max_len=60, start_symbol=TGT.vocab.stoi["<s>"])
    print(out.size())
    #print(out)
    print(batch.trg.data.size())
    #print(batch.trg.data)
    print("Translation:", end="\t")
    for i in range(1, out.size(1)):
        sym = TGT.vocab.itos[out[0, i]]
        if sym == "</s>": break
        print(sym, end =" ")
    print()
    
    print("Target:", end="\t")
    for i in range(1, batch.trg.size(0)):
        print(batch.trg.data[i,0])
        sym = TGT.vocab.itos[batch.trg.data[i, 0]]
        if sym == "</s>": break
        print(sym, end =" ")
    print()
    print()

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


torch.Size([1, 60])
torch.Size([3, 83])
Translation:	0 
Target:	tensor(4, device='cuda:0')
0 tensor(3, device='cuda:0')


torch.Size([1, 60])
torch.Size([3, 83])
Translation:	2 
Target:	tensor(13, device='cuda:0')
5 tensor(3, device='cuda:0')


torch.Size([1, 60])
torch.Size([3, 83])
Translation:	2 
Target:	tensor(13, device='cuda:0')
5 tensor(3, device='cuda:0')


torch.Size([1, 60])
torch.Size([3, 83])
Translation:	2 
Target:	tensor(11, device='cuda:0')
9 tensor(3, device='cuda:0')


torch.Size([1, 60])
torch.Size([3, 83])
Translation:	0 
Target:	tensor(10, device='cuda:0')
1 tensor(3, device='cuda:0')


torch.Size([1, 60])
torch.Size([3, 83])
Translation:	2 
Target:	tensor(11, device='cuda:0')
9 tensor(3, device='cuda:0')


torch.Size([1, 60])
torch.Size([3, 83])
Translation:	2 
Target:	tensor(13, device='cuda:0')
5 tensor(3, device='cuda:0')


torch.Size([1, 60])
torch.Size([3, 83])
Translation:	0 
Target:	tensor(7, device='cuda:0')
4 tensor(3, device='cuda:0')


torch.Size([1, 60]